In [252]:
import collections

import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_federated as tff
import pandas as pd
from sklearn.model_selection import train_test_split

import sys 
import matplotlib 
import scipy as sp 
import IPython
from IPython import display 
import sklearn 
import random
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

TEST_SIZE = 0.2
NUM_CLIENTS = 5
ACTIVE_CLIENTS = 5
BATCH_SIZE = 5
path = os.path.dirname(tff.__file__)
print(path)

-------------------------
/home/ella/Documents/FL/venv-federated/lib/python3.9/site-packages/tensorflow_federated


In [253]:
# Lista delle GPU disponibili
gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibili: ", gpus)

# Verifica se TensorFlow utilizza la GPU
if gpus:
    print("TensorFlow sta usando la GPU")
else:
    print("TensorFlow non sta usando la GPU")

GPUs disponibili:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow sta usando la GPU


In [254]:
# Import del dataset e divisione in train e test
train_df = pd.read_csv('datasets/train.csv')
test_df = pd.read_csv('datasets/test.csv')

# Viene diviso il train set in train e validation set
train_df, val_df = train_test_split(train_df, test_size = TEST_SIZE, random_state = 42)

train_x = train_df.iloc[:,1:].to_numpy()
train_y = train_df.iloc[:,0].to_numpy()

val_x = val_df.iloc[:,1:].to_numpy()
val_y = val_df.iloc[:,0].to_numpy()

test_x = test_df.iloc[:,1:].to_numpy()
test_y = test_df.iloc[:,0].to_numpy()

In [255]:
# Funzione per calcolare la dipendenza tra feature categoriche con il chi-square test.
# Il test ha come ipotesi nulla che le due variabili siano indipendenti. Quindi con un p-value
# minore di 0.05 si rigetta l'ipotesi nulla e si accetta che le due variabili sono dipendenti.
def chi_square(f1, f2):
    crosstab_result=pd.crosstab(index=f1,columns=f2)
    return chi2_contingency(crosstab_result)[1]

In [256]:
# Strategia: Home_planet
# Si nota la forte correlazione tra group ed home planet/destination e quindi per i nan si possono assegnre i valori del gruppo
# (che saranno uguali alla moda). Restano problematici i gruppi composti da una sola persona, si introduce quindi una nuova categoria
# di home planet e destination per persona con origine o destinazione sconosciuta.

def mv_hp_d(dataset):
    print("-------------------------- HomePlanet -------------------------")

    # HomePlanet
    p = chi_square(dataset['HomePlanet'], dataset['Group'])
    print(f"Le feature HomePlanet e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                           else "Le feature Destination e Group non sono correlate")

    print(f"Numero di persone con HomePlanet nullo: {str(dataset['HomePlanet'].isna().sum())}")
    dataset['HomePlanet'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['HomePlanet'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['HomePlanet'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['HomePlanet'].transform(lambda x: x.fillna("unknown"))
    print(f"Numero di persone con HomePlanet nullo: {str(dataset['HomePlanet'].isna().sum())}")

    # Destination
    print("-------------------------- Destination ------------------------")
    p = chi_square(dataset['Destination'], dataset['Group'])
    print(f"Le feature Destination e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                           else "Le feature Destination e Group non sono correlate")
    
    print(f"Numero di persone con Destination nullo: {str(dataset['Destination'].isna().sum())}")
    dataset['Destination'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Destination'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Destination'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['Destination'].transform(lambda x: x.fillna("unknown"))
    print(f"Numero di persone con Destination nullo: {str(dataset['Destination'].isna().sum())}")

# Strategia: Age
# Si distinguono 3 casi di persone con Age nullo: persone sole, persone con famiglia che non spendono e persone con famiglia che spendono.
# Per assegnare l'età si usa la mediana del caso di appartenenza, immaginando che è meno probabile che una persona sola sia un bambino
# e che al contrario è più probabile che una persona che non spende, ma è in un gruppo lo sia.
def mv_age(dataset):
    print("----------------------------- Age -----------------------------")
    solo = dataset['Solo']==1
    family_no_spending = (dataset['Family_size']>1) & (dataset['No_spending']==1)
    family_spending = (dataset['Family_size']>1) & (dataset['No_spending']==0)
    
    print(f"Numero di persone con Age nullo: {str(dataset['Age'].isna().sum())}, di cui:")
    print(f"Persone sole: {str(dataset['Age'][solo].isna().sum())}")
    print(f"Persone con famiglia che non spendono: {str(dataset['Age'][family_no_spending].isna().sum())}")   
    print(f"Persone con famiglia che spendono: {str(dataset['Age'][family_spending].isna().sum())}")

    dataset['Age'][solo] = dataset[solo]['Age'].transform(lambda x: x.fillna(dataset[solo]['Age'].median()))
    dataset['Age'][family_no_spending] = dataset[family_no_spending]['Age'].transform(lambda x: x.fillna(dataset[family_no_spending]['Age'].median()))
    dataset['Age'][family_spending] = dataset[family_spending]['Age'].transform(lambda x: x.fillna(dataset[family_spending]['Age'].median()))

    print(f"Numero di persone con Age nullo: {str(dataset['Age'].isna().sum())}")   


# Strategia: Surname
# Si osserva che i gruppi sono per gran parte composti da persone con lo stesso cognome, quindi si può assegnare la moda del cognome del gruppo
# alle persone con cognome nullo. Per i gruppi composti da una sola persona si assegna cognome 'unknown'.
def mv_surname(dataset):
    print("------------------------------ Surname ------------------------")

    p = chi_square(dataset['Surname'], dataset['Group'])
    print(f"Le feature Surname e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                       else "Le feature Surname e Group non sono correlate")

    print(f"Numero di persone con Surname nullo: {str(dataset['Surname'].isna().sum())}")
    dataset['Surname'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Surname'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Surname'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['Surname'].transform(lambda x: x.fillna("Unknown"))
    print(f"Numero di persone con Surname nullo: {str(dataset['Surname'].isna().sum())}")

# Strategia: RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
# Si sostituiscono i valori nulli con la media di spesa delle altre categorie non nulle, dopo aver osservato che ogni persona ha almeno un valore
# di spesa non nullo. Ad esempio, se una persona ha spesa nulla per RoomService e ShoppingMall, ma ha spesa per FoodCourt e Spa, si assegna la media
# di FoodCourt e Spa per RoomService e ShoppingMall.
def mv_exp(dataset):
    print("------ RoomService, FoodCourt, ShoppingMall, Spa, VRDeck ------")
    print(f"Persone con servizi nulli: {str(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].isnull().any(axis=1).sum())}")
    # Per ogni persona con almeno un servizio nullo, si calcola la media delle spese non nulle
    dataset['RoomService'] = dataset['RoomService'].fillna(dataset[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['FoodCourt'] = dataset['FoodCourt'].fillna(dataset[['RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['ShoppingMall'] = dataset['ShoppingMall'].fillna(dataset[['RoomService', 'FoodCourt', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['Spa'] = dataset['Spa'].fillna(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'VRDeck']].mean(axis=1))
    dataset['VRDeck'] = dataset['VRDeck'].fillna(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa']].mean(axis=1))
    print(f"Persone con servizi nulli: {str(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].isnull().any(axis=1).sum())}")

# Controllare se i gruppi hanno tutti lo stesso valore di cabin_side
def mv_cabins(dataset):
    print("------------------------- Cabin_side --------------------------")
    p = chi_square(dataset['Cabin_side'], dataset['Group'])
    print(f"Le feature Cabin_side e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                          else "Le feature Cabin_side e Group non sono correlate")
    print(f"Numero di persone con Cabin_side nullo: {str(dataset['Cabin_side'].isna().sum())}")
    dataset['Cabin_side'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Cabin_side'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Cabin_side'][(dataset['Solo']==1)] = dataset['Cabin_side'][(dataset['Solo']==1)].fillna('unknown')
    print(f"Numero di persone con Cabin_side nullo: {str(dataset['Cabin_side'].isna().sum())}")

    print("------------------------ Cabin_number -------------------------")
    p = chi_square(dataset['Cabin_number'], dataset['Group'])
    print(f"Le feature Cabin_number e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                            else "Le feature Cabin_number e Group non sono correlate")
    print(f"Numero di persone con Cabin_number nullo: {str(dataset['Cabin_number'].isna().sum())}")
    dataset['Cabin_number'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Cabin_number'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Cabin_number'][(dataset['Solo']==1)] = dataset['Cabin_number'][(dataset['Solo']==1)].fillna('unknown')
    print(f"Numero di persone con Cabin_number nullo: {str(dataset['Cabin_number'].isna().sum())}")
    
    print("------------------------- Cabin_deck --------------------------")
    p = chi_square(dataset['Cabin_deck'], dataset['HomePlanet'])
    print(f"Le feature Cabin_deck e HomePlanet sono correlate, con p-value{p}" if p < 0.05 \
                                                                               else "Le feature Cabin_deck e HomePlanet non sono correlate")
    print(f"Numero di persone con Cabin_deck nullo: {str(dataset['Cabin_deck'].isna().sum())}")
    dataset['Cabin_deck'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('HomePlanet')['Cabin_deck'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Cabin_deck'][(dataset['Solo']==1)] = dataset['Cabin_deck'][(dataset['Solo']==1)].fillna('unknown')
    print(f"Numero di persone con Cabin_deck nullo: {str(dataset['Cabin_deck'].isna().sum())}")

# Strategia: VIP, CrioSleep
# Per le feature VIP e CrioSleep si assegna il valore della moda a tutti i valori nulli.
def mv_vip_crio(dataset):
    print("----------------------------- VIP, CryoSleep ------------------")
    print(f"Numero di persone con VIP nullo: {str(dataset['VIP'].isna().sum())}")
    dataset['VIP'] = dataset['VIP'].fillna(dataset['VIP'].mode()[0])
    print(f"Numero di persone con VIP nullo: {str(dataset['VIP'].isna().sum())}")

    print(f"Numero di persone con CryoSleep nullo: {str(dataset['CryoSleep'].isna().sum())}")
    dataset['CryoSleep'] = dataset['CryoSleep'].fillna(dataset['CryoSleep'].mode()[0])
    print(f"Numero di persone con CryoSleep nullo: {str(dataset['CryoSleep'].isna().sum())}")


In [257]:

def preprocess_data(dataset):
    # Viene aggiunta una colonna per contare il numero di persone nel gruppo di appartenenza e se la persona è da sola
    dataset['Group'] = dataset['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
    dataset['Group_size']=dataset['Group'].map(lambda x: dataset['Group'].value_counts()[x])
    dataset['Solo']=(dataset['Group_size']==1).astype(int)

    # Viene creata una colonna per il cognome e si rimuove la colonna Name
    dataset['Name'].fillna('Unknown Unknown', inplace=True)
    dataset['Surname']=dataset['Name'].str.split().str[-1]
    dataset.loc[dataset['Surname']=='Unknown','Surname']=np.nan
    dataset.drop('Name', axis=1, inplace=True)

    # Vengono riempiti i valori nulli delle colonne RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
    mv_exp(dataset)
    
    # Vengono prese tutte le colonne che rappresentano le spese e si aggiunge una colonna per contare il
    # totale speso dalla persona e una colonna booleana per indicare se la persona non ha speso nulla
    exp_feats=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    dataset['Expenditure']=dataset[exp_feats].sum(axis=1)
    dataset['No_spending']=(dataset['Expenditure']==0).astype(int)

    # Vengono riempiti i valori nulli della colonna HomePlanet, Destination, Surmane
    mv_hp_d(dataset)
    mv_surname(dataset)

    # Viene aggiunta una colonna per contare le persone che hanno lo stesso cognome
    dataset['Family_size']=dataset[(dataset['Surname']!='Unknown')].groupby('Group')['Surname'].transform('count')
    dataset.loc[dataset['Surname']=='Unknown','Family_size']=1
    dataset.loc[dataset['Family_size']>100,'Family_size']=np.nan

    # Vengono riempiti i valori nulli della colonna Age
    mv_age(dataset)


    # Viene aggiunta una colonna che assegna ogni passeggero al gruppo di età di appartenenza
    dataset['Age_group']=0
    dataset.loc[dataset['Age']<=12,'Age_group']='Age_0-12'
    dataset.loc[(dataset['Age']>12) & (dataset['Age']<18),'Age_group']='Age_13-17'
    dataset.loc[(dataset['Age']>=18) & (dataset['Age']<=25),'Age_group']='Age_18-25'
    dataset.loc[(dataset['Age']>25) & (dataset['Age']<=30),'Age_group']='Age_26-30'
    dataset.loc[(dataset['Age']>30) & (dataset['Age']<=50),'Age_group']='Age_31-50'
    dataset.loc[dataset['Age']>50,'Age_group']='Age_51+'

    # Separazione della colonna 'Cabin' nelle colonne deck, number e side
    dataset['Cabin'].fillna('Z/9999/Z', inplace=True)

    dataset['Cabin_deck'] = dataset['Cabin'].apply(lambda x: x.split('/')[0])
    dataset['Cabin_number'] = dataset['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    dataset['Cabin_side'] = dataset['Cabin'].apply(lambda x: x.split('/')[2])

    dataset.loc[dataset['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
    dataset.loc[dataset['Cabin_number']==9999, 'Cabin_number']=np.nan
    dataset.loc[dataset['Cabin_side']=='Z', 'Cabin_side']=np.nan

    dataset.drop('Cabin', axis=1, inplace=True)
    mv_cabins(dataset)
    mv_vip_crio(dataset)


preprocess_data(train_df)
#preprocess_data(val_df)

------ RoomService, FoodCourt, ShoppingMall, Spa, VRDeck ------
Persone con servizi nulli: 691
Persone con servizi nulli: 0
-------------------------- HomePlanet -------------------------
Le feature HomePlanet e Group sono correlate, con p-value7.383208949075836e-101
Numero di persone con HomePlanet nullo: 168
Numero di persone con HomePlanet nullo: 0
-------------------------- Destination ------------------------
Le feature Destination e Group sono correlate, con p-value0.00016495054761429579
Numero di persone con Destination nullo: 139
Numero di persone con Destination nullo: 0
------------------------------ Surname ------------------------
Le feature Surname e Group sono correlate, con p-value0.0
Numero di persone con Surname nullo: 159
Numero di persone con Surname nullo: 0
----------------------------- Age -----------------------------
Numero di persone con Age nullo: 148, di cui:
Persone sole: 87
Persone con famiglia che non spendono: 40
Persone con famiglia che spendono: 21
Nume

In [258]:
# stampare le colonne con i valori nulli
print(train_df.isnull().sum())

PassengerId     0
HomePlanet      0
CryoSleep       0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
Group           0
Group_size      0
Solo            0
Surname         0
Expenditure     0
No_spending     0
Family_size     0
Age_group       0
Cabin_deck      0
Cabin_number    0
Cabin_side      0
dtype: int64
